In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec
import seaborn as sns 
%matplotlib inline 
from matplotlib import rc
rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
from plotly.subplots import make_subplots
import plotly.graph_objects as go
plt.style.use(['ggplot'])

## 주별로 표시하기위해
from math import ceil
from datetime import datetime, timezone, timedelta
pd.options.display.max_rows=1000

#데이터 전처리를 위해
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday
import platform
import missingno as msno
from datetime import datetime, timezone, timedelta
import pytz
from tqdm.notebook import tqdm as tqdm

import warnings
warnings.filterwarnings(action='ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_rows=1000

In [2]:
df = pd.read_csv('과제수정DB.csv')
df.head()

,brandnm,goodsnm,USER,reg_date,order_date,p_type,sub_type,app_yn
0,Gucci,flower dionysus shoulder bag beige,a161237,12/6/19,1/15/21,7days,BA,Y
1,El estilo de Claire,beads cropped jacket black,a1140859,1/2/21,1/15/21,4days,O,NaN
2,Danha,embroidery point oriental mini skirt blue,a1140859,1/2/21,1/15/21,4days,B,NaN
3,Danha,graphics pattern top pink,a1140859,1/2/21,1/15/21,4days,T,NaN
4,Burberry London,buckle basic pattern zipup-jumper navy,a1140740,1/1/21,1/15/21,7days,O,Y


In [3]:
data = df[['brandnm', 'goodsnm', 'sub_type']]

In [4]:
count_vector = CountVectorizer()

In [5]:
c_vector_genres = count_vector.fit_transform(data['goodsnm'])
c_vector_genres = count_vector.fit_transform(data['sub_type'])

In [6]:
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]

In [7]:
gerne_c_sim.shape

(1471, 1471)

In [8]:
def get_recommend_item_list(df, item_set):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_item_index = df[df['goodsnm'] == item_set].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_item_index, :10].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_item_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index][:10]
    return result

In [10]:
get_recommend_item_list(data, item_set='button basic pattern H-Line dress')

,brandnm,goodsnm,sub_type
1470,Burberry,classic trench coat beige,O
484,Fendi,ff logo point cross bag black,BA
485,Chanel,vintage mini cross body,BA
486,Gucci,dionysus cross bag black,BA
487,Saint Laurent,chevron quilted cross bag beige,BA
488,Saint Laurent,chevron quilted cross bag black,BA
489,Becla,pocket detail ribbon belted blouse,T
490,Gucci,logo pointed cross bag,BA
491,Yves Saint Laurent,monogram envelope chain cross bag,BA
